In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
from loadTDMS import *
from tqdm import tqdm
# import seaborn as sns

#### 日付 : 切削中のサンプル数 / last path 
- 20211025 : 5060000 / 24path
- 20211115 : 5010000 / 238path
- 20220630 : 4670000 / 735path
- 20221027 : 4930000 / 168path
- 20230113 : 9970000 / 120path
- 20230117 : 9970000 / (121path ~ 460path) 340path

In [2]:
# date = 20211025
# date = 20211115
# date = 20220630
# date = 20221027
date = 20230113
# date = 20230117

if date == 20211025:
    last_path = 24
    last_sample_num = 5060000
if date == 20211115:
    last_path = 238
    last_sample_num = 5010000
if date == 20220630:
    last_path = 735
    last_sample_num = 4670000 
if date == 20221027:
    last_path = 168
    last_sample_num = 4930000
    
if date == 20230113:
    last_path = 120
    last_sample_num = 9970000

if date == 20230117:
    last_path = 460
    last_sample_num = 9970000


print(rf"date={date}, last_path={last_path}, last_sample_num={last_sample_num}")

date=20230113, last_path=120, last_sample_num=9970000


### new_non_cutting_avg : 非切削時の電圧平均（直線に補間したやつ）
### non_cutting_avg: original 非切削時の電圧平均

In [6]:
ncd_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")
# ncd_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
for num in tqdm(range(121,461)):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
    cd = load_tdms(path, ch_num=3)
    cd = cd[1][:9970000]
    cf = ( cd - ncd_avg[num - 121] ) * (-500)
    np.save(rf"Z:\200 Produced_data\{date}\CF\{num}",cf)

100%|██████████| 340/340 [13:12<00:00,  2.33s/it]


In [132]:
%matplotlib qt
interval = 8308
d1 = np.load(rf"Z:\200 Produced_data\20230113\CF\2.npy")
fig = plt.figure()
plt.plot(d1)
ax = fig.add_subplot()
print(int(len(d1)/interval))

for i in range(int(len(d1)/interval)):
    ax.axvline(i*interval, color = "red")
    i = i+interval

<ipython-input-132-66f4aa35a373>:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot()


1200


In [108]:
d1 = np.load(rf"Z:\200 Produced_data\20230113\CF\2.npy")
N= 2**17
print(N)
d1 = d1[:N]
dt = 1/333333
F = np.fft.fft(d1) # 変換結果
freq = np.fft.fftfreq(N, d=dt) # 周波数
print(freq[1:int(N/2)])

Amp = np.abs(F/(N/2)) # 振幅

fig, ax = plt.subplots()
ax.plot(freq[1:int(N/2)], Amp[1:int(N/2)])
ax.set_xlabel("Freqency [Hz]")
ax.set_ylabel("Amplitude")
ax.grid()
plt.show()

131072
[2.54312897e+00 5.08625793e+00 7.62938690e+00 ... 1.66658871e+05
 1.66661414e+05 1.66663957e+05]


In [ ]:
# 配列Fをコピー
F2 = F.copy()

# ローパスフィル処理（カットオフ周波数を超える帯域の周波数信号を0にする）
F2[(freq > fc)] = 0

# 高速逆フーリエ変換（時間信号に戻す）
f2 = np.fft.ifft(F2)

# 振幅を元のスケールに戻す
f2 = np.real(f2*N)

## 切削抵抗ノイズ除去

In [110]:
# データのパラメータ
N = 2**17            # サンプル数
dt = 1/333333          # サンプリング間隔
fc = 500           # カットオフ周波数
t = np.arange(0, N*dt, dt)  # 時間軸
freq = np.linspace(0, 1.0/dt, N)  # 周波数軸
f = d1[:N]

# 高速フーリエ変換（周波数信号に変換）
F = np.fft.fft(f)

# # 正規化 + 交流成分2倍
# F = F/(N/2)
# F[0] = F[0]/2

# 配列Fをコピー
F2 = F.copy()

# ローパスフィル処理（カットオフ周波数を超える帯域の周波数信号を0にする）
F2[(freq > fc)] = 0

# 高速逆フーリエ変換（時間信号に戻す）
f2 = np.fft.ifft(F2)

# 振幅を元のスケールに戻す
# f2 = np.real(f2*N)

# グラフ表示

fig = plt.figure(figsize=(10.0, 8.0))
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

# 時間信号（元）
plt.subplot(221)
plt.plot(t, f, label='RAW')
plt.xlabel("Time", fontsize=12)
plt.ylabel("Signal", fontsize=12)
plt.grid()
leg = plt.legend(loc=1, fontsize=15)
leg.get_frame().set_alpha(1)

# 周波数信号(元)
plt.subplot(222)
plt.plot(freq, np.abs(F), label='RAW_FFT')
plt.xlabel('Frequency', fontsize=12)
plt.ylabel('Amplitude', fontsize=12)
plt.grid()
leg = plt.legend(loc=1, fontsize=15)
leg.get_frame().set_alpha(1)

# 時間信号(処理後)
plt.subplot(223)
plt.plot(t, f2, label='LPF500Hz')
plt.xlabel("Time", fontsize=12)
plt.ylabel("Signal", fontsize=12)
plt.grid()
leg = plt.legend(loc=1, fontsize=15)
leg.get_frame().set_alpha(1)

# 周波数信号(処理後)
plt.subplot(224)
plt.plot(freq, np.abs(F2), label='LPF500Hz_FFT')
plt.xlabel('Frequency', fontsize=12)
plt.ylabel('Amplitude', fontsize=12)
plt.grid()
leg = plt.legend(loc=1, fontsize=15)
leg.get_frame().set_alpha(1)
plt.savefig('切削抵抗500Hz_lowpass_filter.png')

C:\Users\pmg07\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1317: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
